# Optimize Tensorflow Pipeline Performance (Using prefetch & cache)

In [1]:
import tensorflow as tf
import time

In [2]:
tf.__version__

'2.17.0'

In [6]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # Open file
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)
            
    def __new__(cls):
        print("new called")
        pass

In [7]:
obj = FileDataset()

new called


In [11]:
class FileDataset(tf.data.Dataset):
    def read_files_in_batches(num_samples):
        # Open file
        time.sleep(0.03)
        
        for sample_idx in range(num_samples):
            time.sleep(0.015)
            yield (sample_idx,)
            
    def __new__(cls, num_samples=3):
        return tf.data.Dataset.from_generator(
            cls.read_files_in_batches,
            output_signature=tf.TensorSpec(shape = (1,), dtype = tf.int64),
            args=(num_samples,)
        )

In [12]:
def benchmark(dataset, num_epochs=2):
    for epoch_num in range(num_epochs):
        for sample in dataset:
            time.sleep(0.01)

In [13]:
%%timeit
benchmark(FileDataset())

375 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Using prefetch()

In [14]:
%%timeit
benchmark(FileDataset().prefetch(1))

364 ms ± 17.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
benchmark(FileDataset().prefetch(3))

369 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%%timeit
benchmark(FileDataset().prefetch(tf.data.AUTOTUNE))

375 ms ± 15.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Using cache()

In [19]:
dataset = tf.data.Dataset.range(5)

for d in dataset:
    print(d.numpy())

0
1
2
3
4


In [20]:
dataset = dataset.map(lambda x: x**2)
for d in dataset:
    print(d.numpy())

0
1
4
9
16


In [21]:
dataset = dataset.cache()

for d in dataset.as_numpy_iterator():
    print(d)

0
1
4
9
16


In [22]:
list(dataset.as_numpy_iterator())

[0, 1, 4, 9, 16]

In [24]:
def mapped_function(s):
    tf.py_function(lambda: time.sleep(0.03), [], ())
    return s

In [28]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function), 5)

1.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [29]:
%%timeit -n1 -r1

benchmark(FileDataset().map(mapped_function).cache(), 5)

569 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
